In [2]:
import pandas as pd
import re

# 1. Carga tu CSV (asegúrate de la ruta exacta)
ruta = "Online Retail.csv"
df = pd.read_csv(ruta, encoding="latin1")

# 2. Normaliza la descripción a minúsculas y quita espacios
df["Description_clean"] = df["Description"].str.lower().str.strip()

# 3. Elimina filas donde Description_clean sea NaN o cadena vacía
df = df[df["Description_clean"].notna()]
df = df[df["Description_clean"] != ""]

# 4. Elimina filas donde Quantity o UnitPrice sean cero o negativos
df = df[df["Quantity"] > 0]
df = df[df["UnitPrice"] > 0]

# 5. Diccionario de categorías ↓ keywords
categoria_keywords = {
    "Bolsas":                ["jumbo bag", "lunch bag", "shopper", "storage bag"],
    "Hogar & Decoración":    ["heart", "ornament", "frame", "chalkboard", "holder"],
    "Bakeware":              ["cakestand", "cake tins", "cake cases", "jelly moulds", "jam making"],
    "Iluminación":           ["t-light holder", "lamp", "candle", "light"],
    "Fiestas & Temporada":   ["bunting", "christmas", "decoration"],
    "Cocina & Utensilios":   ["wooden spoon", "cocktail sticks", "recipe box", "bowl", "plate", "jug"],
    "Vajilla & Cristalería": ["teacup and saucer", "glass", "bottle", "mug"],
    "Papelería & Regalos":   ["card", "gift bag", "gift book", "note", "journal"],
    "Almacenamiento":        ["box", "tin", "bag", "storage"],
    "Textiles & Accesorios": ["cushion", "scarf", "apron", "fabric"],
    "Muebles & Accesorios":  ["key holder", "frame", "shelf", "table"],
    "Juguetes & Niños":      ["toy", "puzzle", "doll", "ornament"]
}

# 6. Función para asignar categoría
def asignar_categoria(desc):
    if not isinstance(desc, str):
        return "Otros"
    for categoria, keywords in categoria_keywords.items():
        for kw in keywords:
            if re.search(r"\b" + re.escape(kw) + r"\b", desc):
                return categoria
    return "Otros"

# 7. Aplica la función a la columna limpia
df["Categoria"] = df["Description_clean"].apply(asignar_categoria)

# 8. Muestra las primeras filas con las categorías asignadas
print("Ejemplo de descripciones y sus categorías:")
print(df[["Description", "Categoria"]].drop_duplicates().head(15))

# 9. Agrupa por categoría para resumir
resumen = df.groupby("Categoria").agg({
    "InvoiceNo": "count",
    "Quantity": "sum"
}).rename(columns={"InvoiceNo": "Transacciones", "Quantity": "Total_Unidades"})

print("\nResumen por categoría (Top 10 por unidades vendidas):")
print(resumen.sort_values("Total_Unidades", ascending=False).head(10))


Ejemplo de descripciones y sus categorías:
                            Description              Categoria
0    WHITE HANGING HEART T-LIGHT HOLDER     Hogar & Decoración
1                   WHITE METAL LANTERN                  Otros
2        CREAM CUPID HEARTS COAT HANGER                  Otros
3   KNITTED UNION FLAG HOT WATER BOTTLE  Vajilla & Cristalería
4        RED WOOLLY HOTTIE WHITE HEART.     Hogar & Decoración
5          SET 7 BABUSHKA NESTING BOXES                  Otros
6     GLASS STAR FROSTED T-LIGHT HOLDER     Hogar & Decoración
7                HAND WARMER UNION JACK                  Otros
8             HAND WARMER RED POLKA DOT                  Otros
9         ASSORTED COLOUR BIRD ORNAMENT     Hogar & Decoración
10           POPPY'S PLAYHOUSE BEDROOM                   Otros
11            POPPY'S PLAYHOUSE KITCHEN                  Otros
12    FELTCRAFT PRINCESS CHARLOTTE DOLL       Juguetes & Niños
13              IVORY KNITTED MUG COSY   Vajilla & Cristalería
14   BOX OF 

In [3]:
# Suponiendo que ya hiciste todo lo de asignar Categoria al df
# Por ejemplo, df["Categoria"] = df["Description_clean"].apply(asignar_categoria)

# 1. Elige el nombre del CSV que quieras; aquí le puse “OnlineRetail_Categorias.csv”
df.to_csv("OnlineRetail_Categorias.csv", index=False, encoding="utf-8")

print("¡CSV con categorías generado: OnlineRetail_Categorias.csv!")

¡CSV con categorías generado: OnlineRetail_Categorias.csv!
